In [7]:
# IMPORTING LIBRARIES
import random
import numpy as np

In [8]:
# CONFIG SETUP
suits = ['C','D','H','S']
suits = ["♠","♣","♥","♦" ]
ranks  = {  "J":3,
            "9":2,
            "A":1,"10":1,
            "K":0,"Q":0
}
delim = "|"
num_players=  4
starting_player = 1


In [9]:
#### CARDS & POWER RELATED FUNCTIONS

def cards_distributor(num_players = 4):
    
    cards = [[i+delim+k for k in suits] for i in ranks.keys()]
    cards = [i for j in cards for i in j]
    random.shuffle(cards)
    players_deck = {k+1:list(hand) for k, hand in enumerate(np.array_split(cards,num_players))}
    return players_deck

def deck_power(deck):
    power = np.sum([ranks[i.split(delim)[0]] for i in deck])
    return power

def trump_power(deck):
    trumps = {k:0 for k in suits}
    for card in deck: 
        trumps[card.split(delim)[-1]]=trumps[card.split(delim)[-1]] + 1 + ranks[card.split(delim)[0]]
    return {k:v for k,v  in sorted(trumps.items(), key = lambda x: -x[1])}

In [26]:
#### GAME RELATED FUNCTIONS

def play_order(first_player,num_players):
    
    all_players = list(range(1,num_players+1))
    player_index = all_players.index(first_player)
    return all_players[player_index:] + all_players[:player_index]


def valid_play(round_cards, player_cards, trump = '' ):
    valid_suit =  [i for i in player_cards if i.split(delim)[-1]==round_cards[0].split(delim)[-1]]
    if len(valid_suit)>0: return valid_suit
    trumps = [i for i in player_cards if i.split(delim)[-1]==trump]

    if len(trumps) == 0:
        return player_cards
    return trumps


def next_round_initiator(round_cards, round_order, trump):
    scores = [ranks[card.split(delim)[0]]   for card in round_cards if card.split(delim)[-1] == round_cards[0].split(delim)[-1] ]
    score_map =  dict(zip(round_order, scores))
    score_map =  {k: v for k, v in sorted(score_map.items(), key=lambda item: -item[1])}

    vettu = [(player, card) for (player, card) in zip(round_order, round_cards) if card.split(delim)[-1]==trump]
    vettu = sorted(vettu, key = lambda x: -ranks[x[1].split(delim)[0]])
    if  vettu!=[]:
        return play_order(vettu[0][0], num_players)
    print("NO VETTU")
    # print(list(score_map))
    return play_order(list(score_map)[0], num_players)

def scoring(round_cards, round_order, players_dict):
    vettu = [(player, card) for (player, card) in zip(round_order, round_cards) if card.split(delim)[-1]==trump]
    vettu = sorted(vettu, key = lambda x: -ranks[x[1].split(delim)[0]])
    

    total_score = [ranks[card.split(delim)[0]] for card in round_cards]

    if len(vettu)>0:
        biggest_vettu = vettu[0]
        winner_player = biggest_vettu
    else:
        play_wise_scores = list(zip(round_order, total_score))
        play_wise_scores = sorted(play_wise_scores, key = lambda x: -x[1])
        winner_player = play_wise_scores[0]
        # return 

    print(f"Round Winner: {players_dict[winner_player[0]]['team']} - {sum(total_score)} points")
    return players_dict[winner_player[0]]['team'], sum(total_score)

def game_play(round_cards, play, play_order, players_dict, trump):

    if len(round_cards)==0:
        all_cards = players_dict[play]['cards']
        js = [card for card in all_cards if ranks[card.split(delim)[0]]==3]
        if len(js)>0: return js[0]
    
        kqs = [card for card in all_cards if ranks[card.split(delim)[0]]==0 and card.split(delim)[-1]!=trump]
        if len(kqs)>0: return kqs[0]

        return random.choice(all_cards)

    

    start_suit = round_cards[0].split(delim)[-1]
    pidi_card = sorted(round_cards, key = lambda x: -ranks[x.split(delim)[0]])[0]

    any_trumps = [card for card in round_cards if card.split(delim)[-1]==trump]
    if len(any_trumps)>0: pidi_card = sorted(any_trumps, key = lambda x: -ranks[x.split(delim)[0]])[0]
    

    mapped_cards = list(zip(play_order, round_cards))
    pidi_team = players_dict[[i for i in mapped_cards if i[1]==pidi_card][0]]['team']

    print(pidi_team)

    player_cards = {
        0: [card for card in players_dict[play]['cards'] if card.split(delim)[-1] == start_suit],
        1: [card for card in players_dict[play]['cards'] if card.split(delim)[-1] == trump],
        2: [card for card in players_dict[play]['cards'] if card.split(delim)[-1] not in [start_suit, trump]]
    }

    



In [27]:
print("Starting the Game of 28!")
print("\nDistributing cards")

player_decks = cards_distributor(num_players)
players_dict = {}
for player, deck in player_decks.items():
    players_dict[player] = {} 
    sorted_deck = sorted(deck, key = lambda x: -ranks[x.split(delim)[0]])
    players_dict[player]['cards'] = sorted_deck
    players_dict[player]['deck_power'] = deck_power(deck)
    players_dict[player]['trump_power'] = trump_power(deck)

    if player%2==0: players_dict[player]['team'] = 'B'
    else: players_dict[player]['team'] = 'A'

    print(f" \n Player {player}: {', '.join(sorted_deck)} \t Power: {deck_power(deck)}")
    print(f"\t{trump_power(deck)}")


initial_players_dict = players_dict.copy()

no_of_rounds = int((len(ranks)*len(suits))/num_players)
trump = random.choice(suits)

print(f"TRUMP === {trump}")

players_dict = dict(initial_players_dict)

first_player = starting_player
round_wise_cards = {}
scores_tally = {}
for round_no in range(1,no_of_rounds+1):
    print(f"\n ### ROUND {round_no} ###")
    round_wise_cards[round_no] = {'first_player':starting_player, 'play_order':play_order(first_player, num_players)}
    print(play_order(first_player,num_players))

    round_cards = []
    for play in round_wise_cards[round_no]['play_order']:
        if len(round_cards)==0:
            first_choice = game_play(round_cards, play, round_wise_cards[round_no]['play_order'], players_dict, trump)
            round_cards = [first_choice]
            players_dict[play]['cards'].remove(first_choice)
            # print(round_cards)

        else:
            game_play(round_cards, play, round_wise_cards[round_no]['play_order'], players_dict, trump)
            eligible_play = valid_play(round_cards,players_dict[play]['cards'] , trump)
            # if eligible_play == []: eligible_play = 'TRUMP'
            round_cards.append(eligible_play[0])
        
        
    print(list(zip(round_wise_cards[round_no]['play_order'],round_cards)))
    # print(next_round_initiator(round_cards, play_order(first_player,num_players), trump))
    first_player = next_round_initiator(round_cards, play_order(first_player,num_players), trump)[0]

    team_name, round_score = scoring(round_cards, play_order(first_player,num_players), players_dict)
        

Starting the Game of 28!

Distributing cards
 
 Player 1: J|♦, J|♥, 9|♠, 9|♦, 10|♦, K|♠ 	 Power: 11
	{'♦': 9, '♠': 4, '♥': 4, '♣': 0}
 
 Player 2: J|♣, 10|♣, A|♣, Q|♦, Q|♥, K|♦ 	 Power: 5
	{'♣': 8, '♦': 2, '♥': 1, '♠': 0}
 
 Player 3: J|♠, 9|♣, A|♦, A|♥, K|♥, Q|♣ 	 Power: 7
	{'♠': 4, '♣': 4, '♥': 3, '♦': 2}
 
 Player 4: 9|♥, 10|♥, 10|♠, A|♠, Q|♠, K|♣ 	 Power: 5
	{'♠': 5, '♥': 5, '♣': 1, '♦': 0}
TRUMP === ♥

 ### ROUND 1 ###
[1, 2, 3, 4]


TypeError: 'function' object is not iterable